<h3>Multinomial Naive Bayes</h3>

In [39]:
function counts()
    --gets counts of classes and features
    fc:add(Y:t()*(X))
    cc:add(Y:sum(1))
end

function log_prob()
    --smoothes feature counts
    sfc = torch.add(fc, 1)
    
    --smoothes class counts
    scc = sfc:sum(2)

    --logging
    sfc:log()
    scc:log()
    
    --expands for subtraction compatability 
    scc:expand(scc,scc:size(1),sfc:size(2))
    
    --log probabilities
    lp = sfc:csub(scc)
end   

function log_prior()
    --pseudo prior
    clp:csub(math.log(Y:size(2)))
    
    --actual prior
    --total = cc:sum(2)
    --total:log()
    --total:expand(total,1,cc:size(2))
    --cc:log()
    --clp = cc:csub(total)
end

function predict(X)
    --joint log-likelihood
    local jll = ((X*lp:t()):csub(clp:expand(clp,X:size(1),clp:size(2))))
    return jll
end

In [42]:
--input matrix
X = torch.Tensor{{4.9,  3.1,  1.5,  0.1},
                {6. ,  2.9,  4.5,  1.5},
                { 5.5,  2.6,  4.4,  1.2},
                { 4.8,  3. ,  1.4,  0.3},
                { 5.4,  3.9,  1.3,  0.4},
                { 5.6,  2.8,  4.9,  2. },
       { 5.6,  3. ,  4.5,  1.5},
       { 4.8,  3.4,  1.9,  0.2},
       { 4.4,  2.9,  1.4,  0.2},
       { 6.2,  2.8,  4.8,  1.8},
       { 4.6,  3.6,  1. ,  0.2},
       { 5.1,  3.8,  1.9,  0.4},
       { 6.2,  2.9,  4.3,  1.3},
       { 5. ,  2.3,  3.3,  1. },
       { 5. ,  3.4,  1.6,  0.4},
       { 6.4,  3.1,  5.5,  1.8},
       { 5.4,  3. ,  4.5,  1.5},
       { 5.2,  3.5,  1.5,  0.2},
       { 6.1,  3. ,  4.9,  1.8},
       { 6.4,  2.8,  5.6,  2.2}}

--classes
Y = torch.Tensor{{1, 0, 0},
       {0, 1, 0},
       {0, 1, 0},
       {1, 0, 0},
       {1, 0, 0},
       {0, 0, 1},
       {0, 1, 0},
       {1, 0, 0},
       {1, 0, 0},
       {0, 0, 1},
       {1, 0, 0},
       {1, 0, 0},
       {0, 1, 0},
       {0, 1, 0},
       {1, 0, 0},
       {0, 0, 1},
       {0, 1, 0},
       {1, 0, 0},
       {0, 0, 1},
       {0, 0, 1}}

--test 
test = torch.Tensor{{ 5.8,  2.8 , 5.1,  2.4},
 { 6. ,  2.2 , 4.,   1. },
 { 5.5 , 4.2 , 1.4 , 0.2},
 { 7.3 , 2.9 , 6.3 , 1.8},
 { 5.  , 3.4 , 1.5 , 0.2},
 { 6.3 , 3.3 , 6.  , 2.5},
 { 5.  , 3.5 , 1.3 , 0.3},
 { 6.7 , 3.1 , 4.7 , 1.5},
 { 6.8 , 2.8 , 4.8 , 1.4},
 { 6.1 , 2.8 , 4.  , 1.3},
 { 6.1 , 2.6 , 5.6 , 1.4},
 { 6.4 , 3.2 , 4.5 , 1.5},
 { 6.1 , 2.8 , 4.7 , 1.2},
 { 6.5 , 2.8 , 4.6 , 1.5},
 { 6.1 , 2.9,  4.7 , 1.4},
}

In [41]:
--GLOBAL VARIABLES
--class counts
cc = torch.zeros(1, Y:size(2))
--feature counts
fc = torch.zeros(Y:size(2), X:size(2))
--conditional log probs
lp = torch.zeros(Y:size(1), X:size(2))
--class log probs
clp = torch.zeros(1, Y:size(2))

--fit and calculates \theta
counts()
log_prob()
log_prior()

--\theta
print(lp)

predictions = predict(test)
max, indices = torch.max(predictions, 2)
print(indices)

-0.7396 -1.0976 -1.8766 -3.3269
-0.9295 -1.6026 -1.1991 -2.2790
-0.9804 -1.6959 -1.1521 -2.0759
[torch.DoubleTensor of size 3x4]

 3
 2
 1
 3
 1
 3
 1
 2
 2
 2
 2
 2
 2
 2
 2
[torch.LongTensor of size 15x1]



<h3>Reading in HDF5 files for SST1</h3>

In [20]:
require 'hdf5'

f = hdf5.open("SST1.hdf5", "r")

In [21]:
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_test = f:read("valid_input"):all()
Y_test = f:read("valid_output"):all()

In [11]:
nclasses = f:read('nclasses'):all():long()[1]

In [12]:
nfeatures = f:read('nfeatures'):all():long()[1]

In [13]:
W = torch.DoubleTensor(nclasses, nfeatures)
b = torch.DoubleTensor(nclasses)

In [22]:
f:close()


In [24]:
Y_train:size(1)

156817	


In [25]:
X_train:size(1)

156817	


In [37]:
X_train:sub(100,100)

Columns 1 to 20
 48   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1

Columns 21 to 40
  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1

Columns 41 to 53
  1   1   1   1   1   1   1   1   1   1   1   1   1
[torch.IntTensor of size 1x53]

